In [1]:
import os
import pandas as pd
import numpy as np
import re

In [2]:
DATA_DIR = "/Users/kendra/Data/IMDB/2024-10-05"

## name

In [51]:
file_path = os.path.join(DATA_DIR, "name.basics.tsv")

In [52]:
df1 = pd.read_csv(file_path, sep='\t')

In [53]:
df1.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0083922,tt0050976,tt0069467"


In [9]:
df1.columns

Index(['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession',
       'knownForTitles'],
      dtype='object')

In [66]:
ignore_cols = ['primaryProfession', 'knownForTitles']

rename_dict = {
    'nconst': 'nameId:ID(Person)'
}

for col in ignore_cols:
    rename_dict[col] = f'{col}:IGNORE'

In [12]:
rename_dict

{'nconst': 'nameId:ID(Person)',
 'primaryProfession': 'primaryProfession:IGNORE',
 'knownForTitles': 'knownForTitles:IGNORE'}

In [55]:
df1['is_actor'] = np.where(
    (df1['primaryProfession'].str.contains('actor')) | (df1['primaryProfession'].str.contains('actress')),
    1, 0)
df1['is_director'] = np.where(
    (df1['primaryProfession'].str.contains('director')),
    1, 0)  

df1_filt = df1[(df1['is_actor'] == 1) | (df1['is_director'] == 1)]

In [14]:
n = len(df1)
nf1 = len(df1_filt)
nf1, nf1/n

(6019760, 0.43474919893183206)

## aside: find "bad" nodes

In [54]:
nid = '721526'
df1[df1['nconst'].str.endswith(nid)]

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
679725,nm0721526,Émile Reynaud,1844,1918,"director,animation_department,producer","tt32169968,tt0000003,tt16763740,tt16763674"
1591885,nm10721526,Haris Georgiou,\N,\N,editor,tt7149270
2550000,nm11721526,Trent Jenkins,\N,\N,"director,actor,writer",tt12670402
3490429,nm12721526,Brice Beauer,\N,\N,"actor,writer",tt14961642
4421052,nm13721526,Isabelle Stadelmann-Steffen,\N,\N,\N,tt20255128
5285859,nm14721526,Isobel Onions,\N,\N,costume_designer,tt27385799
6152938,nm15721526,Jane Allison,\N,\N,\N,tt0412116
6984814,nm1721526,Marina Karvas,\N,\N,miscellaneous,tt0420868
7871883,nm2721526,Marian Wright,\N,\N,actress,tt0051307
8767586,nm3721526,Gyeong-soo Choi,\N,\N,miscellaneous,tt0985344


## resume

In [56]:
nid = '721526'
df1_filt[df1_filt['nconst'].str.endswith(nid)]

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,is_actor,is_director
679725,nm0721526,Émile Reynaud,1844,1918,"director,animation_department,producer","tt32169968,tt0000003,tt16763740,tt16763674",0,1
2550000,nm11721526,Trent Jenkins,\N,\N,"director,actor,writer",tt12670402,1,1
3490429,nm12721526,Brice Beauer,\N,\N,"actor,writer",tt14961642,1,0
7871883,nm2721526,Marian Wright,\N,\N,actress,tt0051307,1,0
10508389,nm5721526,Eleanor King,\N,\N,"actress,writer,producer",tt2968700,1,0
12017143,nm7721526,Sadie Weschler,\N,\N,art_director,tt3657200,0,1
13614236,nm9721526,Andres Lopez,\N,\N,actor,tt8184618,1,0


In [59]:
df1_filt = df1_filt.drop(columns=['is_actor', 'is_director'])# 'primaryProfession:IGNORE', 'knownForTitles:IGNORE'])

Remove people who don't have a birth year (imaginary characters?)

In [60]:
df1_filt = df1_filt[df1_filt['birthYear'] != '\\N']

In [17]:
n_filt2 = len(df1_filt)

In [18]:
n_filt2

434118

In [61]:
# replace '\N' with nulls
df1_filt['deathYear'] = np.where(df1_filt['deathYear'] == '\\N', np.nan, df1_filt['deathYear'])

convert id's

In [ ]:
name_id_regex = r'nm0*(\d*)'

df1_filt['nconst'] = df1_filt['nconst'].apply(lambda x: int(re.match(name_id_regex, x).group(1)))

In [63]:
nid = 721526
df1_filt[df1_filt['nconst'] == nid]

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
679725,721526,Émile Reynaud,1844,1918,"director,animation_department,producer","tt32169968,tt0000003,tt16763740,tt16763674"


In [67]:
df1_rename = df1_filt.rename(columns=rename_dict)

In [68]:
df1_rename.sample(10)

,nameId:ID(Person),primaryName,birthYear,deathYear,primaryProfession:IGNORE,knownForTitles:IGNORE
12348208,8141084,Efe Can Yildiz,1990,NaN,"assistant_director,director,writer","tt7081328,tt29765755,tt14587668,tt14587724"
13745179,9874817,Simay Yaykir,1994,NaN,"director,writer,producer",tt8488538
36499,38160,Jack Ary,1919,1974,"actor,writer","tt0054834,tt0061676,tt0133572,tt0057967"
8041331,2917761,Walter Beck,1888,1963,actor,tt1183719
9373780,4431318,Ryan J. Gilmer,1977,NaN,"actor,producer,director","tt7127210,tt6561398,tt7510944,tt4441660"
12720287,8614880,Yui Ooba,1994,NaN,"actress,archive_footage","tt6461280,tt7975206,tt6287718,tt7028802"
861928,916616,Jacques Weber,1949,NaN,"actor,director,camera_department","tt0099334,tt0123808,tt0319041,tt0862998"
109493,115047,Marjorie Browne,1910,1990,actress,"tt0029115,tt0236413,tt0037799,tt0210458"
646718,686257,David Anthony Pizzuto,1951,2012,"actor,location_management,production_manager","tt0172156,tt0260831,tt0248667,tt0455566"
458404,485313,Devinn Lane,1972,NaN,"actress,director,producer","tt0410110,tt0474236,tt0282604,tt0349170"


In [70]:
nid = 721526
df1_rename[df1_rename['nameId:ID(Person)'] == nid]

,nameId:ID(Person),primaryName,birthYear,deathYear,primaryProfession:IGNORE,knownForTitles:IGNORE
679725,721526,Émile Reynaud,1844,1918,"director,animation_department,producer","tt32169968,tt0000003,tt16763740,tt16763674"


In [71]:
nid = 623866
df1_rename[df1_rename['nameId:ID(Person)'] == nid]

,nameId:ID(Person),primaryName,birthYear,deathYear,primaryProfession:IGNORE,knownForTitles:IGNORE
588154,623866,Clara Nebelong,1881,1943,actress,"tt3125484,tt0188039,tt2404004,tt2404012"


In [29]:
df1_rename.to_csv(os.path.join(DATA_DIR, "filt.name.basics.tsv"), sep='\t', header=True, index=False)

In [30]:
del df1, df1_filt, df1_rename

# title basics

In [31]:
file_path = os.path.join(DATA_DIR, "title.basics.tsv")
df = pd.read_csv(file_path, sep='\t')

/var/folders/mp/z98sg8jn1q3bx499xmgkf_040000gn/T/ipykernel_22095/2693477088.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='\t')


In [32]:
df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [33]:
df.columns

Index(['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres'],
      dtype='object')

In [34]:
# ignore_cols = ['primaryProfession', 'knownForTitles']

rename_dict = {
    'tconst': 'titleId:ID(Movie)'
}

# for col in ignore_cols:
#     rename_dict[col] = f'{col}:IGNORE'

# rename_dict

In [35]:
title_id_regex = r'tt0*(\d*)'
df['tconst'] = df['tconst'].apply(lambda x: int(re.match(title_id_regex, x).group(1)))

In [36]:
n = len(df)

In [38]:
df_filt = df[df['isAdult'] == 0]

In [39]:
n_filt = len(df_filt)
n_filt, n_filt/n

(10706453, 0.962146450179395)

In [40]:
df_movies = df_filt[df_filt['titleType'] == 'movie']
len(df_movies)

681857

In [41]:
# replace '\N' with nulls
df_movies['startYear'] = np.where(df_movies['startYear'] == '\\N', np.nan, df_movies['startYear'])
df_movies['runtimeMinutes'] = np.where(df_movies['runtimeMinutes'] == '\\N', np.nan, df_movies['runtimeMinutes'])

/var/folders/mp/z98sg8jn1q3bx499xmgkf_040000gn/T/ipykernel_22095/3649384730.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies['startYear'] = np.where(df_movies['startYear'] == '\\N', np.nan, df_movies['startYear'])
/var/folders/mp/z98sg8jn1q3bx499xmgkf_040000gn/T/ipykernel_22095/3649384730.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies['runtimeMinutes'] = np.where(df_movies['runtimeMinutes'] == '\\N', np.nan, df_movies['runtimeMinutes'])


In [42]:
df_movies.sample(10)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
5476821,21378074,movie,Mandal ke Bol,Mandal ke Bol,0,2020,\N,NaN,\N
9068539,5372998,movie,Mezhdu maskite,Mezhdu maskite,0,2009,\N,54,Documentary
7868710,32645959,movie,Mori: The Christmas Assassin,Mori: The Christmas Assassin,0,NaN,\N,NaN,"Action,Comedy,Crime"
383863,400695,movie,Not Simply a Wedding Banquet,Not Simply a Wedding Banquet,0,1997,\N,50,Documentary
451498,470191,movie,The True Story of Ah Q,Ah Q zheng zhuan,0,1958,\N,109,Drama
384033,400870,movie,Le séga à l'île Maurice,Le séga à l'île Maurice,0,1990,\N,NaN,Documentary
243761,254616,movie,O Olho Mágico do Amor,O Olho Mágico do Amor,0,1982,\N,85,"Comedy,Drama"
10771350,9147564,movie,Hidden Attack of the Dead,Kan-yû,0,2018,\N,74,Horror
451138,469823,movie,Senza paura,Senza paura,0,2000,\N,84,Action
9447405,6229644,movie,The Nation,The Nation,0,2009,\N,NaN,Drama


In [45]:
rename_dict

{'tconst': 'titleId:ID(Movie)'}

In [46]:
rename_dict['primaryTitle'] = 'title'

In [47]:
(df_movies.drop(columns=['isAdult', 'originalTitle', 'endYear', 'titleType'])
     .rename(columns=rename_dict)
    .to_csv(os.path.join(DATA_DIR, "filt.titles.basic.tsv"), sep="\t", header=True, index=False)
)

# check principal (relationship?)

In [3]:
file_path = os.path.join(DATA_DIR, "filt.title.principals.tsv")
df = pd.read_csv(file_path, sep='\t', nrows=10000)

In [4]:
df.head()

,:START_ID(Movie),ordering:IGNORE,:END_ID(Person),category:TYPE,job:IGNORE,characters:IGNORE
0,1,2,5690,director,NaN,NaN
1,2,1,721526,director,NaN,NaN
2,3,1,721526,director,NaN,NaN
3,4,1,721526,director,NaN,NaN
4,5,1,443482,actor,NaN,"[""Blacksmith""]"


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   :START_ID(Movie)   10000 non-null  int64 
 1   ordering:IGNORE    10000 non-null  int64 
 2   :END_ID(Person)    10000 non-null  int64 
 3   category:TYPE      10000 non-null  object
 4   job:IGNORE         123 non-null    object
 5   characters:IGNORE  6028 non-null   object
dtypes: int64(3), object(3)
memory usage: 468.9+ KB
